In [1]:
import datetime
from math import exp, sqrt
import numpy as np

In [2]:
NSIMULATIONS = 1000000

V=0.20 # Volatilidad
R=0.04 # Interés instantáneo (ln(1+r)?)
COUPON_BARRIER=0.8
KICKOUT_BARRIER=1.1
PROTECTION_BARRIER=0.6
COUPON_RATE=0.088

In [3]:
obs_dates = [datetime.datetime(2012, 7, 4, 0, 0),
             datetime.datetime(2013, 7, 5, 0, 0),
             datetime.datetime(2014, 7, 7, 0, 0),
             datetime.datetime(2015, 7, 6, 0, 0),
             datetime.datetime(2016, 7, 5, 0, 0),
             datetime.datetime(2017, 7, 5, 0, 0),
             datetime.datetime(2018, 7, 5, 0, 0)]

# Dates como enteros. Ahorra cálculos innecesarios con objetos datetime. 
dates = [int((t-obs_dates[0]).days) for t in obs_dates]

In [4]:
def new_price(S_t, v, r, t1, t2):
    T = (t2 - t1) / 365.0
    return S_t * exp((r - 0.5 * v * v) * T + v * sqrt(T) * np.random.randn()) # no hay diferencia con gauss


def df(r, t1, t2):
    return exp(-r * (t2 - t1) / 365.0)

In [5]:
def mc_autocall(nsimulations,
                v=V,
                r=R, 
                coupon_barrier=COUPON_BARRIER,
                kickout_barrier=KICKOUT_BARRIER,
                protection_barrier=PROTECTION_BARRIER,
                coupon_rate=COUPON_RATE,
                dates=dates):
    
    assert(coupon_barrier >= protection_barrier)
    
    S = 1.0 
    C = 1000.0 # Inversión inicial

    start_date = dates[0]
    
    autocall_price = 0.0
    
    for i in range(nsimulations):
        payoff = 0.0
        
        for j in range(1, len(dates)):
            S = new_price(S, v, r, dates[j-1], dates[j])
    
            if S > coupon_barrier:
                payoff += C * coupon_rate * df(r, start_date, dates[j])
                
            if S >= kickout_barrier:
                break

        if S < protection_barrier:
            payoff += C * S * df(r, start_date, dates[j])
                
        else:
            payoff += C * df(r, start_date, dates[j])

        autocall_price += payoff

    return float(autocall_price) / float(nsimulations)

In [6]:
%%timeit
mc_autocall(NSIMULATIONS,
            v=V,
            r=R, 
            coupon_barrier=COUPON_BARRIER,
            kickout_barrier=KICKOUT_BARRIER,
            protection_barrier=PROTECTION_BARRIER,
            coupon_rate=COUPON_RATE,
            dates=dates)

3.51 s ± 322 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
mc_autocall(NSIMULATIONS,
            v=V,
            r=R, 
            coupon_barrier=COUPON_BARRIER,
            kickout_barrier=KICKOUT_BARRIER,
            protection_barrier=PROTECTION_BARRIER,
            coupon_rate=COUPON_RATE,
            dates=dates)

1045.1921728353364

In [8]:

def mc_autocall2(nsimulations,
                 v=V,
                 r=R, 
                 coupon_barrier=COUPON_BARRIER,
                 kickout_barrier=KICKOUT_BARRIER,
                 protection_barrier=PROTECTION_BARRIER,
                 coupon_rate=COUPON_RATE,
                 dates=dates):
    
    assert(coupon_barrier >= protection_barrier)
    
    S = 1.0
    C = 1000.0

    start_date = dates[0]

    autocall_prices = []

    coupons_discounted = [C * coupon_rate * df(r, start_date, dates[j]) for j in range(len(dates))]
    principal_discounted = [C * df(r, start_date, dates[j]) for j in range(len(dates))]
    
    for i in range(nsimulations):
        autocall_price = 0.0
        
        for j in range(1, len(dates)):
            S = new_price(S, v, r, dates[j-1], dates[j])
    
            if S > coupon_barrier:
                autocall_price += coupons_discounted[j]
                
            if S >= kickout_barrier:
                break

        if S < protection_barrier:
            autocall_price += S * principal_discounted[j]
                
        else:
            autocall_price += principal_discounted[j]

        autocall_prices.append(autocall_price)

    final_autocall_price = sum(autocall_prices) / float(nsimulations)
    return final_autocall_price

In [9]:
%%timeit
mc_autocall2(NSIMULATIONS,
              v=V,
              r=R, 
              coupon_barrier=COUPON_BARRIER,
              kickout_barrier=KICKOUT_BARRIER,
              protection_barrier=PROTECTION_BARRIER,
              coupon_rate=COUPON_RATE,
              dates=dates)

3 s ± 173 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
def mc_autocall3(nsimulations,
                 v=V,
                 r=R, 
                 coupon_barrier=COUPON_BARRIER,
                 kickout_barrier=KICKOUT_BARRIER,
                 protection_barrier=PROTECTION_BARRIER,
                 coupon_rate=COUPON_RATE,
                 dates=dates):
    
    assert(coupon_barrier >= protection_barrier)
    
    S = 1.0
    C = 1000.0

    start_date = dates[0]

    autocall_prices = []

    coupons_discounted = [C * coupon_rate * df(r, start_date, dates[j]) for j in range(len(dates))]
    principal_discounted = [C * df(r, start_date, dates[j]) for j in range(len(dates))]
    
    def p_exp(t1, t2):
        T = (t2 - t1) / 365.0
        return ((r - 0.5 * v * v) * T, v * sqrt(T)) 
        
    
    partial_exp = [p_exp(dates[j], dates[j+1]) for j in range(len(dates)-1)]
     
    for i in range(nsimulations):
        autocall_price = 0.0
        
        for j in range(1, len(dates)):
            S = S * exp(partial_exp[j-1][0] + partial_exp[j-1][1] * np.random.randn()) 
    
            if S > coupon_barrier:
                autocall_price += coupons_discounted[j]
                
            if S >= kickout_barrier:
                break

        if S < protection_barrier:
            autocall_price += S * principal_discounted[j]
                
        else:
            autocall_price += principal_discounted[j]

        autocall_prices.append(autocall_price)

    final_autocall_price = sum(autocall_prices) / float(nsimulations)
    return final_autocall_price

In [11]:
%%timeit
mc_autocall3(NSIMULATIONS,
             v=V,
             r=R, 
             coupon_barrier=COUPON_BARRIER,
             kickout_barrier=KICKOUT_BARRIER,
             protection_barrier=PROTECTION_BARRIER,
             coupon_rate=COUPON_RATE,
             dates=dates)

1.97 s ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def mc_autocall4(nsimulations,
                 v=V,
                 r=R, 
                 coupon_barrier=COUPON_BARRIER,
                 kickout_barrier=KICKOUT_BARRIER,
                 protection_barrier=PROTECTION_BARRIER,
                 coupon_rate=COUPON_RATE,
                 dates=dates):
    
    assert(coupon_barrier >= protection_barrier)
    
    S = 1.0
    C = 1000.0

    start_date = dates[0]

    autocall_prices = []

    coupons_discounted = [C * coupon_rate * df(r, start_date, dates[j]) for j in range(len(dates))]
    principal_discounted = [C * df(r, start_date, dates[j]) for j in range(len(dates))]
    
    def p_exp(t1, t2):
        T = (t2 - t1)/ 365.0
        return ((r - 0.5 * v**2) * T, v * sqrt(T)) 
        
    
    partial_exp = [p_exp(dates[j], dates[j+1]) for j in range(len(dates)-1)]
    
    rnd = np.random.randn(nsimulations, len(dates)-1)   
    
    for i in range(nsimulations):
        autocall_price = 0.0
        
        for j in range(1, len(dates)):
            S = S * exp(partial_exp[j-1][0] + partial_exp[j-1][1] * rnd[i, j-1]) 
    
            if S > coupon_barrier:
                autocall_price += coupons_discounted[j]
                
            if S >= kickout_barrier:
                break

        if S < protection_barrier:
            autocall_price += S * principal_discounted[j]
                
        else:
            autocall_price += principal_discounted[j]

        autocall_prices.append(autocall_price)

    final_autocall_price = sum(autocall_prices) / float(nsimulations)
    return final_autocall_price

In [13]:
%%timeit
mc_autocall4(NSIMULATIONS,
             v=V,
             r=R, 
             coupon_barrier=COUPON_BARRIER,
             kickout_barrier=KICKOUT_BARRIER,
             protection_barrier=PROTECTION_BARRIER,
             coupon_rate=COUPON_RATE,
             dates=dates)

1.83 s ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
def mc_autocall5(nsimulations,
                 v=V,
                 r=R, 
                 coupon_barrier=COUPON_BARRIER,
                 kickout_barrier=KICKOUT_BARRIER,
                 protection_barrier=PROTECTION_BARRIER,
                 coupon_rate=COUPON_RATE,
                 dates=dates):
    
    assert(coupon_barrier >= protection_barrier)
    
    S = 1.0
    C = 1000.0

    start_date = dates[0]

    autocall_prices = [0.0] * nsimulations

    coupons_discounted = [C * coupon_rate * df(r, start_date, dates[j]) for j in range(len(dates))]
    principal_discounted = [C * df(r, start_date, dates[j]) for j in range(len(dates))]
    
    def p_exp(t1, t2):
        T = (t2 - t1) / 365.0
        return ((r - 0.5 * v**2) * T, v * sqrt(T)) 
        
    
    partial_exp = [p_exp(dates[j], dates[j+1]) for j in range(len(dates)-1)]
    
    rnd = np.random.randn(nsimulations, len(dates)-1)   
    
    for i in range(nsimulations):
        autocall_price = 0.0
        
        for j in range(1, len(dates)):
            S = S * exp(partial_exp[j-1][0] + partial_exp[j-1][1] * rnd[i, j-1]) 
    
            if S > coupon_barrier:
                autocall_price += coupons_discounted[j]
                
            if S >= kickout_barrier:
                break

        if S < protection_barrier:
            autocall_price += S * principal_discounted[j]
                
        else:
            autocall_price += principal_discounted[j]

        autocall_prices[i] = autocall_price

    final_autocall_price = sum(autocall_prices) / float(nsimulations)
    return final_autocall_price

In [15]:
%%timeit
mc_autocall5(NSIMULATIONS,
             v=V,
             r=R, 
             coupon_barrier=COUPON_BARRIER,
             kickout_barrier=KICKOUT_BARRIER,
             protection_barrier=PROTECTION_BARRIER,
             coupon_rate=COUPON_RATE,
             dates=dates)

1.79 s ± 46.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from multiprocessing import Pool

def mc_autocall_mapper(nsimulations,
                   v=V,
                   r=R, 
                   coupon_barrier=COUPON_BARRIER,
                   kickout_barrier=KICKOUT_BARRIER,
                   protection_barrier=PROTECTION_BARRIER,
                   coupon_rate=COUPON_RATE,
                   dates=dates):
    
    assert(coupon_barrier >= protection_barrier)

    # Hay que tener cuidado con la desserialización desde json. 
    # En json son todo flotantes.
    nsimulations = int(nsimulations)
    
    S = 1.0
    C = 1000.0

    start_date = dates[0]

    autocall_prices = [0.0] * nsimulations

    coupons_discounted = [C * coupon_rate * df(r, start_date, dates[j]) for j in range(len(dates))]
    principal_discounted = [C * df(r, start_date, dates[j]) for j in range(len(dates))]
    
    def p_exp(t1, t2):
        T = (t2 - t1) / 365.0
        return ((r - 0.5 * v**2) * T, v * sqrt(T)) 
        
    
    partial_exp = [p_exp(dates[j], dates[j+1]) for j in range(len(dates)-1)]
    
    rnd = np.random.RandomState().randn(nsimulations, len(dates)-1) 
    
    # muy importante RandomState() para multiproceso ya que reinicializa la semilla
    # numpy.random.RandomState(seed=None)
    # If seed is None, then the MT19937 BitGenerator is initialized by reading data from /dev/urandom 
    # (or the Windows analogue) if available or seed from the clock otherwise.
    
    for i in range(nsimulations):
        autocall_price = 0.0
        
        for j in range(1, len(dates)):
            S = S * exp(partial_exp[j-1][0] + partial_exp[j-1][1] * rnd[i, j-1]) 
    
            if S > coupon_barrier:
                autocall_price += coupons_discounted[j]
                
            if S >= kickout_barrier:
                break

        if S < protection_barrier:
            autocall_price += S * principal_discounted[j]
                
        else:
            autocall_price += principal_discounted[j]

        autocall_prices[i] = autocall_price

    return sum(autocall_prices), float(nsimulations)

In [22]:

nworkers = 2 # Por simplicidad debe ser divisible 
nsims = float(NSIMULATIONS) / float(nworkers)

p = Pool(nworkers)


chunks = [int(nsims)] * nworkers
diff = NSIMULATIONS - int(nsims) * nworkers
if diff>0:
    for i in range(diff):
        chunks[i] += 1

# Lo dejamos por completitud.
# En linux funciona incluso en los notebooks de Jupyter
def f(nsimulations):
    return mc_autocall_mapper(nsimulations,
                          v=V,
                          r=R, 
                          coupon_barrier=COUPON_BARRIER,
                          kickout_barrier=KICKOUT_BARRIER,
                          protection_barrier=PROTECTION_BARRIER,
                          coupon_rate=COUPON_RATE,
                          dates=dates)

# En windows desde los notebooks de Jupyter da error
# la serialización de funciones en multiprocessing.
# El truco es escribir la función en un módulo y pasar la referencia.

import mc_autocall_mapper

f = mc_autocall_mapper.f

def map_reduce(f, n):
    suma, size = list(zip(*p.map(f, chunks)))
    return sum(suma)/float(sum(size))
    

In [23]:
%%timeit
map_reduce(f, nworkers)


1.06 s ± 62.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
map_reduce(f, nworkers)

1045.1810666292934